In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from unicodedata import normalize
from datetime import datetime
import time

---

# DATA CLEASING

In [2]:
cadastro = pd.read_csv('dados_case_ds2\\cadastrais.csv', parse_dates=True)
tpv = pd.read_csv('dados_case_ds2\\tpv-mensais-treinamento.csv')

- **comentátios:**

Antes de iniciarmos um EDA, verifica-se a existência de id's duplicados na base, o que prejudicará futuras análises e cruzamento entre as bases. Assumindo id único para estabelicimento, pode-se inferir que houve uma atualização cadastral e, assim, definimos que o último dado corresponde a informação mais precisa.

Contudo, ao verificarmos com os dados de TPV mensal, nota-se o início do relacionamento entre o estabelecimento e a Stone. Para os dados verificados, em geral esse relacionamento se iniciou na data mais antiga.

Dessa forma, agrupa-se os dados para corrigir essas inconsistências. Outros casos podem surgir e serão discutidos futuramente, como por exemplo nos casos em que o assumimos o início do relacionamento com a primeira data, mas apenas existem dados de transações referente-se à segunda data.

In [3]:
duplicated_ids = cadastro['id'][cadastro['id'].duplicated()].values

In [4]:
cadastro[cadastro['id'].isin(duplicated_ids)].sort_values(['id', 'StoneCreatedDate']).head(4)

,id,StoneCreatedDate,StoneFirstTransactionDate,MCC,MacroClassificacao,segmento,sub_segmento,persona,porte,TPVEstimate,tipo_documento,Estado
166484,21,2018-06-19 00:00:00.000,20180626.0,5995.0,Varejo,Petshops e Veterinários,Petshops,SMB - Pequeno Porte e Ticket Medio,10k-25k,3000.0,PJ,BA
130681,21,2020-04-07 00:00:00.000,20210102.0,5912.0,Supermercado/Farmácia,"Farmácias, Produtos de Saúde e Cosmética",Drogarias e Farmácias,SMB - Pequeno Porte e Ticket Baixo,10k-25k,10000.0,PJ,BA
185808,29,2015-02-19 00:00:00.000,20150221.0,7230.0,Serviços,Serviços de Beleza e Estética,Salão de Beleza,SMB - Medio Porte e Ticket Medio,25k-50k,0.0,PJ,DF
237704,29,2017-02-21 00:00:00.000,20170223.0,7230.0,Serviços,Serviços de Beleza e Estética,Salão de Beleza,SMB - Medio Porte e Ticket Medio,50k-100k,66000.0,PJ,DF


- **comentátios:**
    
Exemplos de dados duplicados:

In [5]:
cadastro[cadastro['id'] == 83099].sort_values(['id', 'StoneCreatedDate'])

,id,StoneCreatedDate,StoneFirstTransactionDate,MCC,MacroClassificacao,segmento,sub_segmento,persona,porte,TPVEstimate,tipo_documento,Estado
174299,83099,2017-12-11 00:00:00.000,20171212.0,8043.0,Bens duráveis,Moda e Acessórios,Óticas e Óculos,SMB - Medio Porte e Ticket Alto,50k-100k,55200.0,PJ,RJ
123981,83099,2020-04-16 00:00:00.000,0.0,8043.0,Bens duráveis,Moda e Acessórios,Óticas e Óculos,SMB - Medio Porte e Ticket Alto,50k-100k,10000.0,PJ,RJ


In [6]:
tpv[tpv['id'] == 83099]

,id,mes_referencia,TPV_mensal
1360035,83099,20171231,19190.76
1360036,83099,20180131,49969.58
1360037,83099,20180228,36602.90
1360038,83099,20180331,54438.33
1360039,83099,20180430,53653.76
1360040,83099,20180531,32458.55
1360041,83099,20180630,52948.00
1360042,83099,20180731,63286.10
1360043,83099,20180831,44664.80
1360044,83099,20180930,31218.50


In [7]:
cadastro[cadastro['id'] == 178808].sort_values(['id', 'StoneCreatedDate'])

,id,StoneCreatedDate,StoneFirstTransactionDate,MCC,MacroClassificacao,segmento,sub_segmento,persona,porte,TPVEstimate,tipo_documento,Estado
93886,178808,2015-01-27 00:00:00.000,20150305.0,5499.0,Alimentação,Alimentação e Bebidas,Comércio de Alimentos,Outro,2.5k-5k,0.0,PJ,MG
103844,178808,2020-01-28 00:00:00.000,20200204.0,5499.0,Alimentação,Alimentação e Bebidas,Comércio de Alimentos,Outro,2.5k-5k,0.0,PJ,MG


In [8]:
tpv[tpv['id'] == 178808]

,id,mes_referencia,TPV_mensal
2616135,178808,20200229,773.65
2616136,178808,20200331,2476.91
2616137,178808,20200430,5864.95
2616138,178808,20200531,6457.70
2616139,178808,20200630,6874.45
2616140,178808,20200731,5335.04


---

In [9]:
cadastro = cadastro.sort_values(['id', 'StoneCreatedDate', 'StoneFirstTransactionDate'])
cadastro_df1 = cadastro[['id', 'StoneCreatedDate', 'StoneFirstTransactionDate']]
cadastro_df2 = cadastro.drop(['StoneCreatedDate', 'StoneFirstTransactionDate'], axis='columns')

In [10]:
cadastro_df1 = cadastro_df1.drop_duplicates(subset=['id'], keep='first')
cadastro_df2 = cadastro_df2.drop_duplicates(subset=['id'], keep='last')

In [11]:
cadastro_proc = cadastro_df1.merge(cadastro_df2, on='id', how='inner')
cadastro_proc = cadastro_proc[cadastro_proc['id'].isin(tpv['id'].drop_duplicates())]

In [12]:
cadastro_proc = cadastro_proc[cadastro_proc['id'].isin(tpv['id'].drop_duplicates())]

In [13]:
cadastro_proc['id'] = cadastro_proc['id'].astype(int)
cadastro_proc = cadastro_proc.set_index('id')

In [14]:
cadastro_proc['StoneCreatedDate'] = pd.to_datetime(cadastro_proc['StoneCreatedDate'], yearfirst=True, dayfirst=False)
cadastro_proc['StoneFirstTransactionDate'] = cadastro_proc['StoneFirstTransactionDate'].astype(int)

In [15]:
cadastro_proc.loc[cadastro_proc['StoneFirstTransactionDate'] != 0., 'StoneFirstTransactionDate'] = pd.to_datetime(cadastro_proc.loc[cadastro_proc['StoneFirstTransactionDate'] != 0., 'StoneFirstTransactionDate'], format='%Y%m%d')
cadastro_proc.loc[cadastro_proc['StoneFirstTransactionDate'] == 0., 'StoneFirstTransactionDate'] = pd.to_datetime(cadastro_proc.loc[cadastro_proc['StoneFirstTransactionDate'] == 0., 'StoneCreatedDate'], yearfirst=True, dayfirst=False)

In [16]:
cadastro_proc['StoneFirstTransactionDate'] = pd.to_datetime(cadastro_proc['StoneFirstTransactionDate'], yearfirst=True, dayfirst=False)

- **comentátios:**

Seguindo com a análise dos dados, verifica-se a necessidade de redefinir os estabelecimentos que foram atribuídos como MCC (Merchant Category Code) igual a zero. Nesse caso, iremos recuperar os dados definidos à priori. Nesse caso, assumi-se erro no preenchimento dos dados.

In [17]:
cadastro_proc[cadastro_proc['MCC'].astype(int) == 0]

,StoneCreatedDate,StoneFirstTransactionDate,MCC,MacroClassificacao,segmento,sub_segmento,persona,porte,TPVEstimate,tipo_documento,Estado
id,,,,,,,,,,,
295,2015-11-11,2015-12-02,0.0,NaN,NaN,NaN,Micro Empreendedor,0-2.5k,0.0,PJ,NaN
1405,2019-01-08,2019-02-18,0.0,NaN,NaN,NaN,Micro Empreendedor,0-2.5k,0.0,PJ,NaN
1794,2018-09-13,2018-09-13,0.0,NaN,NaN,NaN,Micro Empreendedor,0-2.5k,0.0,PJ,NaN
1900,2019-07-01,2019-08-07,0.0,NaN,NaN,NaN,Outro,50k-100k,0.0,PJ,NaN
2087,2018-12-17,2018-12-21,0.0,NaN,NaN,NaN,Micro Empreendedor,0-2.5k,0.0,PJ,NaN
...,...,...,...,...,...,...,...,...,...,...,...
201999,2017-05-08,2017-05-09,0.0,NaN,NaN,NaN,Outro,0-2.5k,0.0,PJ,NaN
202613,2016-06-23,2016-06-25,0.0,NaN,NaN,NaN,Micro Empreendedor,0-2.5k,0.0,PJ,NaN
205297,2016-11-11,2016-11-23,0.0,NaN,NaN,NaN,Micro Empreendedor,0-2.5k,0.0,PJ,NaN


In [18]:
cadastro.loc[cadastro['id'] == 202613]

,id,StoneCreatedDate,StoneFirstTransactionDate,MCC,MacroClassificacao,segmento,sub_segmento,persona,porte,TPVEstimate,tipo_documento,Estado
66951,202613,2016-06-23 00:00:00.000,20160625.0,5651.0,Varejo,Moda e Acessórios,Vestuário,SMB - Medio Porte e Ticket Medio,25k-50k,0.0,PJ,RS
94766,202613,2016-06-23 00:00:00.000,20170811.0,5651.0,Varejo,Moda e Acessórios,Vestuário,SMB - Medio Porte e Ticket Medio,25k-50k,1.0,PJ,RS
233660,202613,2018-03-08 00:00:00.000,0.0,0.0,NaN,NaN,NaN,Micro Empreendedor,0-2.5k,0.0,PJ,NaN


In [19]:
cadastro_proc.loc[[202613]]

,StoneCreatedDate,StoneFirstTransactionDate,MCC,MacroClassificacao,segmento,sub_segmento,persona,porte,TPVEstimate,tipo_documento,Estado
id,,,,,,,,,,,
202613,2016-06-23,2016-06-25,0.0,NaN,NaN,NaN,Micro Empreendedor,0-2.5k,0.0,PJ,NaN


In [20]:
tpv[tpv['id'] == 202613]

,id,mes_referencia,TPV_mensal
3007957,202613,20170731,47825.37
3007958,202613,20170831,56675.67
3007959,202613,20170930,46128.42
3007960,202613,20171031,49422.26
3007961,202613,20171130,46588.80
3007962,202613,20171231,69155.55
3007963,202613,20180131,45996.89
3007964,202613,20180228,42857.66
3007965,202613,20180331,49910.95
3007966,202613,20180430,36444.21


In [21]:
cadastro_proc.loc[(cadastro_proc['MCC'].astype(int) == 0)&(cadastro_proc['StoneFirstTransactionDate'] != 0.), 'StoneFirstTransactionDate']

id
295      2015-12-02
1405     2019-02-18
1794     2018-09-13
1900     2019-08-07
2087     2018-12-21
            ...    
201999   2017-05-09
202613   2016-06-25
205297   2016-11-23
205469   2018-08-10
205963   2017-03-24
Name: StoneFirstTransactionDate, Length: 334, dtype: datetime64[ns]

In [22]:
cadastro_mcc_zero = cadastro[cadastro['id'].isin(cadastro_proc.loc[cadastro_proc['MCC'].astype(int) == 0].index)].sort_values(['id', 'StoneCreatedDate', 'StoneFirstTransactionDate', 'MCC']).drop_duplicates(subset=['id'], keep='first').set_index('id')
cadastro_mcc_zero.loc[cadastro_mcc_zero['StoneFirstTransactionDate'] != 0., 'StoneFirstTransactionDate'] = pd.to_datetime(cadastro_mcc_zero.loc[cadastro_mcc_zero['StoneFirstTransactionDate'] != 0., 'StoneFirstTransactionDate'].astype(int), format='%Y%m%d')
cadastro_proc.loc[cadastro_proc['MCC'].astype(int) == 0] = cadastro_mcc_zero
cadastro_proc.loc[(cadastro_proc['MCC'].astype(int) == 0)&(cadastro_proc['StoneFirstTransactionDate'] != 0.), 'StoneFirstTransactionDate'] = pd.to_datetime(cadastro_proc.loc[(cadastro_proc['MCC'].astype(int) == 0)&(cadastro_proc['StoneFirstTransactionDate'] != 0.), 'StoneFirstTransactionDate'], format='%Y%m%d')

In [23]:
id_nan = cadastro_proc[cadastro_proc['StoneFirstTransactionDate'] == 0].index
cadastro_id_nan = cadastro.set_index('id').loc[id_nan]
cadastro_id_nan['NaN qtd'] = np.sum(cadastro_id_nan.replace({0.0:np.nan}).isna(), axis='columns')
cadastro_id_nan = cadastro_id_nan.reset_index().sort_values(['NaN qtd']).drop_duplicates(subset='id', keep='first').set_index('id').sort_index().drop('NaN qtd', axis='columns')
cadastro_id_nan.loc[cadastro_id_nan['StoneFirstTransactionDate'] != 0., 'StoneFirstTransactionDate'] = pd.to_datetime(cadastro_id_nan.loc[cadastro_id_nan['StoneFirstTransactionDate'] != 0., 'StoneFirstTransactionDate'].astype(int), format='%Y%m%d')
cadastro_proc[cadastro_proc['StoneFirstTransactionDate'] == 0.] = cadastro_id_nan

In [24]:
cadastro_proc.loc[cadastro_proc['StoneFirstTransactionDate'] == 0, 'StoneFirstTransactionDate'] = pd.to_datetime(cadastro_proc.loc[cadastro_proc['StoneFirstTransactionDate'] == 0, 'StoneCreatedDate'])
cadastro_proc['StoneCreatedDate'] = pd.to_datetime(cadastro_proc['StoneCreatedDate'], yearfirst=True, dayfirst=False)

In [25]:
cadastro_proc['StoneFirstTransactionDate'] = pd.to_datetime(cadastro_proc['StoneFirstTransactionDate'])

In [26]:
cadastro_proc

,StoneCreatedDate,StoneFirstTransactionDate,MCC,MacroClassificacao,segmento,sub_segmento,persona,porte,TPVEstimate,tipo_documento,Estado
id,,,,,,,,,,,
1,2018-12-12,2018-12-13,5942.0,Varejo,"Livrarias, Papelarias, Gráficas e Bancas",Livrarias e Papelarias,SMB - Medio Porte e Ticket Medio,50k-100k,71000.0,PJ,MG
2,2020-04-13,2020-04-14,5231.0,Bens duráveis,Materiais de Construção,Materiais de Construção,SMB - Pequeno Porte e Ticket Alto,10k-25k,10001.0,PF,RS
3,2019-05-31,2019-06-15,5411.0,Supermercado/Farmácia,Supermercados,Supermercados,Pequeno Empreendedor,2.5k-5k,3000.0,PF,RJ
4,2017-09-29,2017-10-02,4215.0,Serviços,Logística e Mobilidade,Delivery e Entrega,SMB - Medio Porte e Ticket Medio,25k-50k,25000.0,PJ,RJ
5,2019-01-31,2019-01-31,5698.0,Serviços,Serviços de Beleza e Estética,Salão de Beleza,SMB - Pequeno Porte e Ticket Medio,5k-10k,6000.0,PF,PE
...,...,...,...,...,...,...,...,...,...,...,...
206326,2019-05-16,2019-05-20,7997.0,Serviços recorrentes,Esporte e Fitness,Academias e Clubes,SMB - Pequeno Porte e Ticket Alto,10k-25k,4000.0,PF,SP
206327,2020-01-21,2020-01-22,5331.0,Bens duráveis,Lojas de Departamento,Lojas de Departamento,SMB - Pequeno Porte e Ticket Alto,10k-25k,20000.0,PJ,PE
206328,2019-07-15,2019-07-17,8021.0,Serviços,Serviços de Saúde e Clínicas,Odontologia,Pequeno Empreendedor,2.5k-5k,2000.0,PF,RJ


In [27]:
estados = {'Acre':'AC',
            'Alagoas':'AL',
            'Amapa':'AP',
            'Amazonas':'AM',
            'Bahia':'BA',
            'Ceara':'CE',
            'Espirito Santo':'ES',
            'Goias':'GO',
            'Maranhao':'MA',
            'Mato Grosso':'MT',
            'Mato Grosso do Sul':'MS',
            'Minas Gerais':'MG',
            'Para':'PA',
            'Paraiba':'PB',
            'Parana':'PR',
            'Pernambuco':'PE',
            'Piaui':'PI',
            'Rio de Janeiro':'RJ',
            'Rio Grande do Norte':'RN',
            'Rio Grande do Sul':'RS',
            'Rondonia':'RO',
            'Roraima':'RR',
            'Santa Catarina':'SC',
            'Sao Paulo':'SP',
            'Sergipe':'SE',
            'Tocantins':'TO',
            'Distrito Federal':'DF', 
            'NAN':'BR'}

In [28]:
cadastro_proc['MCC'] = cadastro_proc['MCC'].astype(int).astype(str).str.zfill(4)
cadastro_proc['MacroClassificacao'] = cadastro_proc['MacroClassificacao'].fillna('Outro')
cadastro_proc['segmento'] = cadastro_proc['segmento'].fillna('Outro')
cadastro_proc['sub_segmento'] = cadastro_proc['sub_segmento'].fillna('Outro')
cadastro_proc['persona'] = cadastro_proc['persona'].fillna('Outro')
cadastro_proc['porte'] = cadastro_proc['porte'].fillna('Outro')
cadastro_proc['TPVEstimate'] = cadastro_proc['TPVEstimate'].astype(int).fillna(0)
cadastro_proc['tipo_documento'] = cadastro_proc['tipo_documento'].fillna('Outro')
cadastro_proc['Estado'] = cadastro_proc['Estado'].apply(lambda x: normalize('NFKD', str(x)).encode('ASCII', 'ignore').decode('ASCII'))
cadastro_proc['Estado'] = cadastro_proc['Estado'].replace(estados).str.upper()

In [29]:
assert cadastro_proc.isna().sum().sum() == 0

In [30]:
cadastro_proc.to_csv('preprocessed data//cadastrais_preprocessed.csv')

---

In [34]:
tpv['mes_referencia'] = pd.to_datetime(tpv['mes_referencia'], format='%Y%m%d')

In [38]:
tpv.to_csv('preprocessed data//tpv_mensal_preprocessed.csv', index=False)

---